In [1]:
import tree.ctutils as ctu
from tree import treeutils
import numpy as np
import pickle

# Calculate merger event parameters
def find_merger(atree, idx=None, aexp_min=0.0):
    """
        find indices of merger event from a tree.
        (Full tree or main progenitor trunk)
    """
    if idx == None:
        idx = atree['id'][0]

    nprg = 1
    merger_list=[]

    i = 0
    while nprg > 0:
        idx = ctu.get_progenitors(atree, idx, main=True)[0]
        ind = np.where(atree['id'] == idx)[0]
        if atree['aexp'][ind] < aexp_min:
            break
        nprg = ctu.get_npr(atree, idx)

        if nprg > 1:
            merger_list.append(i)
        i +=1
    return merger_list

def merger_mass_ratio(atree, idx=None):
    """
    return mass ratio of the given merger event
    """
    if idx == None:
        idx = atree['id'][0]

    prgs = ctu.get_progenitors(atree, idx)

    # only for mergers
    if len(prgs) > 1:
        i_prgs = [np.where(atree['id'] == i)[0] for i in prgs]
        mass = []
        for iprg in i_prgs:
            mass.append(atree['m'])
    else:
        print("This is not a merger")
        return 0

def merger_properties_main_prg(atree, idx):
    """
        Calculate merger mass ratio for "one" merger event.

    if idx == None:
        if nout == None:
            print("Both idx and nout are missing")
            return
    else:
        if nout == None:
            nout = np.where(atree['id'] == idx)[0]

    idx = atree['id'][ind]
    """

    #prgs = get_progenitors(atree, idx)
    #if len(prgs) > 1:
    #    i_prgs = [np.where(atree['id'] == i)[0] for i in prgs]

    i_prgs = np.where(atree['desc_id'] == idx)[0]

    print(i_prgs)
    id_prgs = atree['id'][i_prgs]
    mass_prgs = atree['m'][i_prgs]

    #mass_prgs_norm = mass_prgs / sum(mass_prgs)

    return mass_prgs

def load_tree(wdir, is_gal=False, no_dump=False):
    import pickle
    from tree import treemodule
    import tree.ctutils as ctu

    alltrees = treemodule.CTree()


    if is_gal:
        # Galaxy tree
        tree_path = 'GalaxyMaker/Trees/'
    else:
        # halo tree
        tree_path = 'halo/Trees/'

    try:
        alltrees = pickle.load(open(wdir + tree_path + "extended_tree.pickle", "rb" ))
        print("Loaded an extended tree")
    except:
        alltrees = treemodule.CTree()
        alltrees.load(filename= wdir + tree_path + 'tree_0_0_0.dat')
        if not no_dump:
            # Fix nout -----------------------------------------------------
            nout_max = alltrees.data['nout'].max()
            alltrees.data['nout'] += 187 - nout_max
            print("------ NOUT fixed")
            alltrees.data = ctu.augment_tree(alltrees.data, wdir, is_gal=is_gal)
            print("------ tree data extended")

    return alltrees


In [7]:
import utils.match as mtc
import matplotlib.pyplot as plt
import pandas as pd

is_gal = True
nout_fi = 187

#Last merger
import matplotlib.pyplot as plt
nout_ini = 67 # recent merger = nout = 140 or later.

# Load tree
is_gal = True

# all catalogs
verbose=False

#
most_recent_only = False

clusters = ['39990', '36415', '10002', '05427', '36413', '01605']

# final result arrays
gal_list=[]
mr_list=[]
nout_list=[]

#for cluster in clusters:
wdir = '/home/hoseung/Work/data/' + clusters[3] + '/'
#wdir = './'

alltrees = load_tree(wdir, is_gal=is_gal)

ft = alltrees.data[alltrees.data['nout'] == nout_fi]
# Galaxies with final mass over 10^10 Msun.
allgals = ft['id'][ft['m'] > 1e10]
print(allgals)

Loaded an extended tree
[123037 122668 122676 123047 122680 123087 122669 122683 122664 122750
 122672 122714 122684 122667 122751 122701 123054 122671 122665 122679
 122745 123032 122675 122773 123072 122682 122737 122753 122928 122666
 122814 122670 122690 122763 122673 122784 122706 122697 122696 122687
 122685 122733 122739 122709 122693 122756 122694 122718 122727 122754
 122674 122677 122835 122722 122695 122698 122747]


### Merger trees of allgals.

In [8]:
#allgals = [183337]
verbose=False
for idx in sorted(allgals):
    #gal = cat['id']
    #if verbose: print("analyzing merger events of galaxy ", gal)

    # Convert halo id to tree id
    #idx = id2idx(alltrees.data, gal, 187)
    #idx = cat['idx']

    # full tree of a galaxy
    atree = ctu.extract_a_tree(alltrees.data, idx)

    # main progenitor tree
    main = ctu.extract_main_tree(atree, idx)

    x_nout = main['nout'].flatten()
    x_nout = x_nout[x_nout > nout_ini]

    mass_ratios_single = np.zeros(len(x_nout))
    for i, nout in enumerate(x_nout):
        # merger ratio
#        print("nout = {}, main = {}".format(nout, main['id'][i]))
#        i_nout_now = np.where(atree['nout'] == nout)[0]
#        print(atree['desc_id'][i_nout_now], atree['m'][i_nout_now])
        i_prgs = np.where(atree['desc_id'] == main['id'][i])[0]
        
        
        # multiple prgs = merger
        if len(i_prgs) > 1:
            if verbose: print(" {} mergers at nout = {}".format(len(i_prgs), nout))
            id_prgs = atree['id'][i_prgs]
            mass_prgs = atree['m'][i_prgs]
            m_r = mass_prgs / max(mass_prgs)
            if verbose:
#                print(mass_prgs)
                print(" Mass ratios : ", m_r)
            mass_ratios_single[i] = max([mass_prgs[1:] / max(mass_prgs)][0])
        else:
            mass_ratios_single[i] = 0

    ind_ok = np.where(mass_ratios_single > 0.1)[0]
#    print("all ind_ok",idx, ind_ok)
    if len(ind_ok) > 0:
        # if a satellite oscillates around the host, 
        # it could be identified as multiple mergers with short time interval. 
        # leave only the first passage / merger.
        good =[]
        for i in range(len(ind_ok)-1):
            if ind_ok[i+1] > ind_ok[i] + 2:
                good.append(ind_ok[i])
        good.append(ind_ok[-1])
        ind_ok = good
#        if most_recent_only:
#            ind_ok = max(ind_ok) # most recent 

#        print("  galaxy {}, Last nout {}, Merger ratio 1:{:.1f}".format(idx,
#                                                                     x_nout[ind_ok],
#                                                                       1./mass_ratios_single[ind_ok]))
        mr = 1./mass_ratios_single[ind_ok]

        gal_list.append(idx)
        mr_list.append(mr)
        nout_list.append(x_nout[ind_ok])

In [74]:
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

nout_fi = 187
nout_ini = 37
nouts = np.arange(nout_ini, nout_fi + 1)
nnouts = len(nouts)
"""
for inout, nout in enumerate(reversed(np.arange(nout_ini, nout_fi + 1))):
    #cat = load_pickle(wdir + 'catalog_GM/' + 'catalog' + str(nout) + '.pickle')
    cat = pickle.load(open(wdir + 'catalog_GM/' + 'catalog' + str(nout) + '.pickle', 'rb'))
    #final_gals.update(cat['final'])
    #print(cat)
    final_gals = cat['idx']
"""
#final_gals = [main['id'][0]]
for final_gals in gal_list:
    #ngals = len(final_gals)
    mstar = np.zeros(nnouts)
    l_r = np.zeros(nnouts)
    reff = np.zeros(nnouts)
    fg = np.zeros(nnouts, dtype=int)
    #final_gals = np.sort(list(final_gals))

#%%#######################################################################
# Read catalogs and extract mstar and lambda-r of galaxies at all nouts.
    idx = final_gals
    atree = ctu.extract_a_tree(alltrees.data, idx)
    main = ctu.extract_main_tree(atree, idx)
    for inout, nout in enumerate(nouts):
        cat = load_pickle(wdir + 'catalog_GM/' + 'catalog' + str(nout) + '.pickle')
        for igal, idgal in enumerate(cat['idx']):
            #ind = which galaxy..? -> need tree. (save the ID of final halo too)
            imain = 0
            mnow = main[main['nout'] == nout]
            ind = np.where(idgal == mnow['id'])[0]
            #print(mnow['id'],ind, final_gals[ind], inout, nout, idgal, cat['id'][igal])
            if len(ind) > 0 :
                fg[inout] = mnow['id']
                mstar[inout] = cat['mstar'][ind]
                l_r[inout] = cat['lambda_r'][ind]
                reff[inout] = cat['rgal'][ind]

    fig, ax = plt.subplots(2)
    ax[0].plot(nouts, np.log10(mstar))
    ax[1].plot(nouts, l_r)
    for i in nout_list[1]:
        ax[0].axvline(x=i, linestyle=':')
        ax[1].axvline(x=i, linestyle=':')

    ax[0].set_ylabel("log10(M*)")
    ax[1].set_ylabel("Rotation parameter")
    #ax[0].scatter(nout_list[1], mr_list[1])
    plt.savefig(wdir + "mstar_lambda_merger_" + str(idx) + '.png')

/home/hoseung/.local/lib/python3.4/site-packages/matplotlib/pyplot.py:424: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


KeyboardInterrupt: 

In [70]:
wdir

'/home/hoseung/Work/data/05427/'